author: Lavender
tag: Research
title: Laplace Redux : Effortless Bayesian Deep Learning 리뷰
abstract: 2021 년 NeurIPS의 Laplace Redux 논문을 리뷰해 봅시다!
image: lap.png

# Bayesian Neural Network 복습
안녕하세요! 코르카의 라벤더입니다.
이번 포스트를 통해 2021년 NeurIPS에서 Cambridge, Tubingen 연구진들이 보인 Laplace approximation 기술이 뭔지 알아보겠습니다. 그 전에, Bayesian inference 부터 살펴보도록 해요. 첫 포스트인 만큼 친절하게 진행하겠습니다.

## Bayesian Inference

간단한 NN 은 하나의 input 에 대해 하나의 output 을 내놓습니다. 한번의 학습 과정을 통해 하나의 weight를 얻고요.
기본적인 Gradient descent 를 통해 데이터를 추론하는 과정은, 주어진 데이터와 모델 weight 를 보고, "이 모델이 데이터를 얼마나 잘 설명하는지" 에 대한 정도를 계산하고 업데이트 하는 과정입니다. 흔히 이 일열의 과정을 "Maximum Likelihood Estimation" 이라고 합니다.

$$
\theta^* = \arg \max_{\theta} p(D | \theta)
$$

그리고 $p(D | \theta)$ 를 evaluate 하는 과정 자체가 model 의 하나의 output 을 보고 평가하는 과정과 대응됩니다.

한편 이걸로 충분하지 않을 수 있습니다. 저희가 모델에 대한 분포를 어떻게 가정하냐에 따라 다른 결과가 나올 수 있습니다. 그리고, 계산 결과물의 초점과 objective 가 하나의 $\theta$ 를 구하는 것으로 귀결됩니다. 따라서, "모델의 정확도" 등을 생각하자면 다른 접근이 필요합니다. 저희는 때로 주어진 데이터를 보고 데이터가 얼마나 $\theta$ 를 잘 설명하는지 생각하고 싶습니다. 엥? 데이터가 $\theta$ 를 얼마나 잘 설명하는지, $\theta$ 가 데이터를 얼마나 잘 생각하는지 뭐가 다르냐고요? 맞습니다. $\theta$ 에 대한 prior (우리가 '기대'하는 $\theta$ 의 분포) 이 없다면 이 둘은 사실 같은 말입니다.

하지만, $\theta$ 에 대한 distribution 이 있다면 다른 이야기입니다. 이런 상황을 생각해봅시다. 동전이 앞면이 나올 확률을 $\theta$ 라 해봅시다. 저희가 동전을 두 번 던져 앞면 앞면이 나왔다면, $\theta = 1$ 일 때 데이터를 잘 설명할 것입니다. 한편, $\theta$ 에 대한 저희의 사전 분포 (Prior) 가 있다면요? 즉, 저희는 $\theta \sim 0.5$ 라는 사실을 알고 있기에 데이터가 $\theta = 1$ 인 상황을 잘 설명하지 않겠죠. *prior* distribution ($\theta$ 에 대한 distribution : $\theta \sim N(0.5, \sigma^2)$ 같은거겠죠?) 이 있을 때, 데이터(동전들) 을 보고 $\theta$ 에 대한 distribution 을 업데이트 하는 과정처럼 보인다는 것을 확인하셨을 겁니다. 묻고 싶은 대상이 $\theta$ 일 때, 저흰 데이터를 보고 업데이트 된 이 분포를 *posterior* 이라 합니다. 

| 처음부터 왜 *굳이 굳이* $\theta$ 의 distribution 을 생각하는거죠? 

여러 장점이 있습니다만, 중요한 장점 중 하나는 *predictive distribution* 을 생각할 수 있다는 겁니다. 이런 상황을 생각해 봅시다. NN 이 만든 prediction $NN(x) = y$ 에 대해, $y$ 의 uncertainty 계산하고 싶으면 어떻게 하는게 좋을까요? 딱히 좋은 방법이 떠오르지 않을 것입니다. 한편, $\theta$ 의 distribution 이 있다면 자연스럽게 $y$ 의 distribution 을 계산하여 $y$ 의 uncertainty 를 논할 수 있습니다. 더 설명드리자면, input $x$ 가 정해질 때 $y$ 의 distribution 을 저희가 계산한 posterior $\theta$ 로 marginalize 하는 것입니다. 즉,

$$
P(y | x, D) = \int_{\theta} P(y | x, \theta) P(\theta, D) d\theta
$$

물론 이 intergral 은 계산이 불가능하기에, monte-carlo 하게 계산하곤 합니다.



# Laplace Approximation


그럼 저희의 목표는 $P(\theta, D)$ 를 구하는 것이죠. 한편 이런 생각을 할 수 있지 않을까요?

**$P(\theta | D)$ 는 사실 $N(\theta_{\text{best}}, \Sigma)$ 이지 않을까?**

상당한 주장입니다. MAP 로 구한 $\theta_{\text{best}}$ 근방의 모델만을 기반으로 $P(\theta, D)$ 를 근사하겠다는 생각이죠. 더 멀리 떨어진 $\theta$ 는 모델의 학습 과정이 만든 것으로 불충분하다는 생각입니다. 그럼 "어떤 근방" 을 "어떻게" 구하여 사용한다는 건가요? 이는 $p$ 가 exponetial family 라는 것을 착안하여, "가장 적합한 가우시안" 으로 생각합니다. 

*참고 :왜 Exponential family 냐구요?*

$$
P(D | \theta) \propto \exp (\sum_{x, y} l(f_\theta(x), y))
$$

꼴로 모델링합니다. $l(\hat{y},y) $ 는 어떤 criterion (Loss function) 이고요, 여기에 gaussian prior 를 자주 사용하기에 결국

$$
P(\theta | D) \propto \exp (\sum_{x, y} l(f_\theta(x), y)) - \gamma \sum_{\theta} \theta^2)
$$

등의 모습이 됩니다.


그래서 자주 $\exp$ 로 나타나는건 알겠고, 이걸 많이 생략해서 $q(\theta) := P(\theta | D) = 1/ Z \exp (\sum_{x, y} l(f_\theta(x), y)) - \gamma \sum_{\theta} \theta^2) := 1/Z \exp(h(\theta))$ 로 생각합시다.

이제 $h(\theta)$ 를 *이차 테일러 근사* 해봅시다

$$
h(\theta) = h(\theta_{\text{best}}) + \frac{1}{2} (\theta - \theta_{\text{best}})^T \nabla^2_{theta} h |_{\theta_{\text{best}}} (\theta - \theta_{\text{best}})
$$

오잉? 왜 일차항은 없어졌죠? 맞습니다. 저흰 $\theta_{\text{best}}$ 가 최적이라는 가정을 쓴 겁니다. 만약 일차항이 존재한다면 그 방향으로 가면 되니까요. 이차항 밖에 없습니다.

이제 가우시안의 pdf 를 볼까요? 가우시안은:

$$
\exp(-\frac{1}{2} (\theta - \theta_{\text{best}})^T \Sigma^{-1} (\theta - \theta_{\text{best}}))
$$

처럼 생겼습니다. 어? 둘이 비슷하네요~ 앞에 있는 값은 normalizing term $Z$ 로 빠진다고 생각하고, 저 이차항 사이에 있는 값만 맞추면 되겠습니다. 그럼 저희의 approximation 은 다음과 같아집니다. 이를 *Laplace Approximation* 이라 합니다.

$$
p(\theta | D) \sim N(\theta, \theta_{\text{best}} \Sigma^{-1}), \Sigma = -\nabla^2_\theta L(D, \theta) |^{-1}_{\theta_{\text{best}}}
$$
